In [182]:
import exifread
import os
import datetime
from rtree import index
import pyproj
import csv

In [276]:
#converGPStoNYLISP
#converts GPS DMS coordinates to coordinates
#in the New York - Long Island State Plane Projection Coordinates
def convertGPStoNYLISPC (GPSLong,GPSLat):
    Degs,Mins,Secs = (0,1,2)
    Long = []
    Lat=[]
    #data Degs Mins and Secs are Ratios
    #convert them to floats
    Lat.append(float(GPSLat[Degs].num))
    Lat.append(float(GPSLat[Mins].num))
    Lat.append(float(GPSLat[Secs].num / GPSLat[Secs].den))
    
    Long.append(float(GPSLong[Degs].num))
    Long.append(float(GPSLong[Mins].num))
    Long.append(float(GPSLong[Secs].num / GPSLong[Secs].den))
    
    degLat  = Lat[Degs] + (60*Lat[Mins] + Lat[Secs]) / (60*60)
    degLong = Long[Degs] + (60*Long[Mins] + Long[Secs]) / (60*60)
    
    #convert to New York - Long Island State Plane Cordinates (ESRI:102718)
    projectstr = "+proj=lcc +lat_1=40.66666666666666 +lat_2=41.03333333333333 +lat_0=40.16666666666666 +lon_0=-74 +x_0=300000 +y_0=0 +ellps=GRS80 +datum=NAD83 +to_meter=0.3048006096012192 +no_defs"
    p = pyproj.Proj(projectstr,preserve_units=True)
    
    xcoord,ycoord = p(-degLong,degLat)  
    return (xcoord,ycoord)

In [288]:
#create rtree index for project list
#PRE: project_keys is a list of (Borough,Block,Lot) pairs
#Borough, Block and Lot are strings

#Post: Returns an rtree index for project_lst
#
def createProjectIndex(project_keys):
    plutofiles = ['BK.csv',  'BX.csv',  'MN.csv',  'QN.csv',  'SI.csv']
    proj_index = index.Rtree('proj_index')
    idx =0
    for file in plutofiles:
        with open(file,'r') as borough_file:
            borough_reader = csv.DictReader(borough_file)
            for lot in borough_reader:
                key = (lot['Borough'],lot['Block'],lot['Lot'])
                if key in project_keys:
                    #get lot coordinates
                    xcoord = int(lot['XCoord'])
                    ycoord = int(lot['YCoord'])
                    
                    #find bounding box around lot
                    radius = 400
                    left,right = xcoord - radius, xcoord + radius 
                    bottom,top = ycoord - radius, ycoord + radius
                    
                    #insert key in the rtree index
                    print("inserting idx=" + str(idx) + " left= " + str(left) + " bottom = " + str(bottom) +" right= " + str(right) + "top= " + str(top))
                    proj_index.insert(idx,(left,bottom,right,top),obj=key)
                    
                    #remove index from the list
                    project_keys.remove(key)
                    if len(project_keys) == 0:
                        proj_index.close()
                        return
                    
                    #increment index
                    idx+=1
    
    #WARNING if this stage is reached there must beARE's  a wrong entry
    #in the project key or the pluto let the user know
    print("the following projects were not found:")
    for pr in project_keys:
        print(pr)
    
    proj_index.close()
    return 

In [282]:
#insert images in the project folder
#PRE thre is a folder named 'proj_index' that
#with all the projcets
def movePicturesToFolders():
    file_list = os.listdir()
    proj_index = index.Rtree('proj_index')
    
    if not os.path.exists('insert_manually'):
        os.mkdir('insert_manually')
    
    for image in file_list:
        try:
            #try extrating the exif info
            with open(image,'rb') as f:
                #read the exif tags
                tags = exifread.process_file(f)
                xcoord,ycoord = convertGPStoNYLISPC (tags['GPS GPSLongitude'].values,\
                                                    tags['GPS GPSLatitude'].values)
                #find collitions
                intersections = [lot.object for lot in proj_index.intersection((xcoord,ycoord,xcoord,ycoord),objects=True)]
                if len(intersections) != 1:
                    if len(intersection) > 1:
                        #if multiple candidates put it in the folder of the colsest property
                        mindist=10000000
                        minid=0
                        for i in intersections:
                            #here I need to get the coordinates for the job
                            #TODO add coorditaes to the rtree object so we can
                            #do this comparison
                            #also read documentation for nearest function
                            #it may be easier to just use that
                    
                    #move to special folder for images tha must be moved manually
                    #os.rename(image,os.path.join('insert_manually',image))
                    print(intersections)
                    #print("could not find xcoord: " + str(xcoord) + " ycoord: " + str(ycoord))
                else:
                    directory = intersections[0][0] + "_" + intersections[0][1] + "_" + intersections[0][2]
                    #make directory to move the images
                    if not os.path.exists(directory):
                        os.mkdir(directory)
                    #move image to folder with name = object
                    #os.rename(image,os.path.join(directory,image))
                    print("Moving: " + image + "to " + os.path.join(directory,image))
        except:
            print("EXIF could not be extracted from " + image)
            
    proj_index.close()
    return
        

In [187]:
#read projcets from file
#assume file project is named proj_file and is in csv format
#proj_file most be a csv file with required fields Borough, Block, Lot
def readProjectKeys():
    keys = []
    with open('proj_file','r') as proj_file:
        reader = csv.DictReader(proj_file)
        for project in reader:
            #create keys
            key = (project['Borough'],project['Block'],project['Lot'])
            keys.append(key)
    
    return keys

In [ ]:
#pluto dataset
#keys of interest: Borough,Block,Lot,Address,XCoord,YCoord
#XCoord and YCoord

#image EXIF
#keys of interest: 

In [286]:
mykeys = readProjectKeys()
mykeys
createProjectIndex(mykeys)

inserting idx=0 left= 1007173 bottom = 193085 right= 1008773top= 194685
inserting idx=1 left= 1009718 bottom = 239745 right= 1011318top= 241345
inserting idx=2 left= 1020747 bottom = 236166 right= 1022347top= 237766
inserting idx=3 left= 992281 bottom = 215362 right= 993881top= 216962
inserting idx=4 left= 1006554 bottom = 210782 right= 1008154top= 212382
inserting idx=5 left= 1006615 bottom = 210538 right= 1008215top= 212138
inserting idx=6 left= 1006118 bottom = 211363 right= 1007718top= 212963
inserting idx=7 left= 1005699 bottom = 211151 right= 1007299top= 212751
inserting idx=8 left= 1007517 bottom = 216986 right= 1009117top= 218586
inserting idx=9 left= 1009447 bottom = 221438 right= 1011047top= 223038
inserting idx=10 left= 1017769 bottom = 212911 right= 1019369top= 214511
inserting idx=11 left= 1030003 bottom = 198231 right= 1031603top= 199831
inserting idx=12 left= 1044721 bottom = 222351 right= 1046321top= 223951
inserting idx=13 left= 1049121 bottom = 217428 right= 1050721to

In [287]:
movePicturesToFolders()

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[('QN', '148', '20'), ('QN', '148', '80'), ('QN', '154', '44')]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[('QN', '148', '20'), ('QN', '148', '80'), ('QN', '154', '44'), ('QN', '182', '93')]
[]
[]
[]
[]
Moving: IMG_1739.JPGto QN_1452_44/IMG_1739.JPG
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[('QN', '148', '20'), ('QN', '148', '80'), ('QN', '154', '44')]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
Moving: IMG_1735.JPGto QN_1452_44/IMG_1735.JPG
[]
[]
[]
[('QN', '148', '20'), ('QN', '148', '80'), ('QN', '154', '44'), ('QN', '182', '93')]
[]
[]
[]
[]
[]
[]
[]
[]
[]
Moving: IMG_1421.JPGto QN_6314_1/IMG_1421.JPG
[]
[]
Moving: IMG_1734.JPGto QN_1452_44/IMG_1734.JPG
Moving: IMG_1410.JPGto QN_6314_1/IMG_1410.JPG
[]
[]
[]
EXIF could not be extracted from MN.csv
[]
[]
[]
Moving: IMG_1464.JPGto QN_3348_77/IMG_1464.JPG
[]
[]
[]
[]
[]
[]
[]
[]
[]
EXIF could not be extracted from proj_index.dat
[]
[]
[]
[]
[]
[]
[]
[]
EXIF could not be extracted from QN_6314_1
[]
[]
[]
[]
[]
Moving: IMG_